In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Model ,models, layers, optimizers, utils

## Reconstruction LSTM Autoencoder

The simplest LSTM autoencoder is one that learns to reconstruct each input sequence.
For these demonstrations, we will use a dataset of one sample of nine time steps and one feature:

In [18]:
# define input sequence
sequence = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

# reshape input into [samples, timesteps, features]
n_in = len(sequence)
sequence = sequence.reshape((1, n_in, 1))

In [4]:
# define model
model = models.Sequential()
model.add(layers.LSTM(100, activation='relu', input_shape=(n_in, 1)))
model.add(layers.RepeatVector(n_in))
model.add(layers.LSTM(100, activation='relu', return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(1)))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(sequence, sequence, epochs=300, verbose=0)

In [5]:
# predict
yhat = model.predict(sequence)
yhat

array([[[0.10559099],
        [0.20217314],
        [0.30041453],
        [0.39952287],
        [0.49908453],
        [0.5987617 ],
        [0.69832975],
        [0.7991052 ],
        [0.9024458 ]]], dtype=float32)

## Prediction LSTM Autoencoder

We can modify the reconstruction LSTM Autoencoder to instead predict the next step in the sequence.
In the case of our small contrived problem, we expect the output to be the sequence:

In [6]:
# define input sequence
seq_in = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

# reshape input into [samples, timesteps, features]
n_in = len(seq_in)
seq_in = seq_in.reshape((1, n_in, 1))

# prepare output sequence
seq_out = seq_in[:, 1:, :]
n_out = n_in - 1

In [7]:
# define model 
model = models.Sequential()
model.add(layers.LSTM(100, activation='relu', input_shape=(n_in, 1)))
model.add(layers.RepeatVector(n_out))
model.add(layers.LSTM(100, activation='relu', return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(1)))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(seq_in, seq_out, epochs=300, verbose=0)

In [8]:
# predict
yhat = model.predict(seq_in)
yhat

array([[[0.16683361],
        [0.2898971 ],
        [0.403169  ],
        [0.5089176 ],
        [0.6094323 ],
        [0.7060289 ],
        [0.7997408 ],
        [0.89148134]]], dtype=float32)

## Composite LSTM Autoencoder

Finally, we can create a composite LSTM Autoencoder that has a single encoder and two decoders, one for reconstruction and one for prediction.

In [9]:
# define input sequence
seq_in = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

# reshape input into [samples, timesteps, features]
n_in = len(seq_in)
seq_in = seq_in.reshape((1, n_in, 1))

# prepare output sequence
seq_out = seq_in[:, 1:, :]
n_out = n_in - 1

In [10]:
# define encoder
visible = layers.Input(shape=(n_in, 1))
encoder = layers.LSTM(100, activation='relu')(visible)
# define reconstruct decoder
decoder1 = layers.RepeatVector(n_in)(encoder)
decoder1 = layers.LSTM(100, activation='relu', return_sequences=True)(decoder1)
decoder1 = layers.TimeDistributed(layers.Dense(1))(decoder1)
# define predict decoder
decoder2 = layers.RepeatVector(n_out)(encoder)
decoder2 = layers.LSTM(100, activation='relu', return_sequences=True)(decoder2)
decoder2 = layers.TimeDistributed(layers.Dense(1))(decoder2)
# concat model
model = Model(inputs=visible, outputs=[decoder1, decoder2])
model.compile(optimizer='adam', loss='mse')
# utils.plot_model(model, show_shapes=True, to_file='composite_lstm_autoencoder.png')

# fit model 
model.fit(seq_in, [seq_in, seq_out], epochs=300, verbose=0)

In [11]:
# predict
yhat = model.predict(seq_in)
yhat

[array([[[0.10127164],
         [0.19949059],
         [0.29943317],
         [0.39987874],
         [0.50023794],
         [0.60028654],
         [0.7000689 ],
         [0.79983366],
         [0.89999163]]], dtype=float32), array([[[0.19868489],
         [0.30206183],
         [0.3981459 ],
         [0.4989811 ],
         [0.600592  ],
         [0.7013527 ],
         [0.80077535],
         [0.8988221 ]]], dtype=float32)]

## Keep Standalone LSTM Encoder

Regardless of the method chosen (reconstruction, prediction, or composite), once the autoencoder has been fit, the decoder can be removed and the encoder can be kept as a standalone model.

The encoder can then be used to transform input sequences to a fixed length encoded vector.

We can do this by creating a new model that has the same inputs as our original model, and outputs directly from the end of encoder model, before the RepeatVector layer.

In [12]:
# define input sequence
seq_in = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

# reshape input into [samples, timesteps, features]
n_in = len(seq_in)
seq_in = seq_in.reshape((1, n_in, 1))

In [15]:
# define model
model = models.Sequential()
model.add(layers.LSTM(100, activation='relu', input_shape=(n_in,1)))
model.add(layers.RepeatVector(n_in))
model.add(layers.LSTM(100, activation='relu', return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(1)))
model.compile(optimizer='adam', loss='mse')

In [16]:
# fit model
model.fit(sequence, sequence, epochs=300, verbose=0)
# connect the encoder LSTM as the output layer
model = Model(inputs=model.inputs, outputs=model.layers[0].output)

In [17]:
# predict
yhat = model.predict(sequence)
print(yhat.shape)
print(yhat)

(1, 100)
[[0.00362787 0.         0.         0.08213381 0.08537362 0.0432427
  0.         0.         0.01888101 0.         0.0003575  0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.13369964
  0.         0.03780872 0.03652276 0.08293831 0.         0.
  0.05843791 0.07145996 0.         0.         0.03048124 0.08965353
  0.         0.06230371 0.         0.08197825 0.         0.
  0.         0.01264344 0.06010371 0.06315333 0.         0.
  0.         0.         0.         0.10529362 0.08009851 0.09936683
  0.         0.09390101 0.         0.         0.07640777 0.
  0.         0.05778015 0.         0.         0.12206501 0.
  0.         0.         0.         0.         0.         0.11213464
  0.         0.04824944 0.         0.14111644 0.07408844 0.
  0.         0.         0.         0.         0.06998165 0.
  0.         0.0079558  0.08975866 0.         0.         0.
  0.09860425 0.         0.         0.         0.0327

## \#What is the difference between 'return_sequences=Ture' and 'RepeatVector'?
- "<code>return_sequence=True</code>" returns all the outputs the encoder observed in the past.
- "<code>RepeatVector</code>" repeats the very last output of the encoder
<img src='https://i.stack.imgur.com/LNXjF.jpg'>


## \#What is the TimeDistributed layers?
- Using TimeDistributed layer
<img src="https://mblogthumb-phinf.pstatic.net/MjAxOTA3MTlfMTQ0/MDAxNTYzNDk5MDIwNjk0.Ko1jG4ematFNFGaS7dFJqJCKoyIVhLXsSLsUNWzadukg.laQainx0gqsofM7EmEi-A5POshd0OkX4yC4Ay0ZeOkwg.GIF.chunjein/2-1.gif?type=w2">

- Not using TimeDistributed layer(= return_sequences=True)
<img src="https://mblogthumb-phinf.pstatic.net/MjAxOTA3MTlfMjEz/MDAxNTYzNDk5MzU3MTU3.CvAWXC9qs5JiguevobzkDPmOggpTNGJNhNJvbh7bEy0g.uTUc7Hbx-UIXsP3tRZVMGFDj_7wv4YVwokvwkIwKlgwg.GIF.chunjein/2-2.gif?type=w2">